In [244]:
import matplotlib.pyplot as plt
import numpy as np
import random as r
import math

In [245]:
class data_preprocessing:
    def __init__(self,instance_path):
        self.instance_path=instance_path
        self.info, self.flights = self.read_file(f_name=self.instance_path)
        
        self.flights_by_day_dict = self.flights_by_day(flight_list=self.flights)
        self.list_days= self.flights_by_day_dict.keys()
        self.airports_by_area = self.get_airports_by_areas()
        
        self.number_of_areas,self.starting_airport=int(self.info[0][0]),self.info[0][1]
        self.starting_area=self.associated_area_to_airport(airport=self.starting_airport)
        
        self.visited_areas=[self.associated_area_to_airport(self.starting_airport)]
        
    def read_file(self,f_name):
        dist = []
        line_nu = -1
        with open(f_name) as infile:
            for line in infile:
                line_nu += 1
                if line_nu == 0:
                    index = int(line.split()[0]) * 2 + 1
                if line_nu >= index:
                    temp = line.split()
                    temp[2] = int(temp[2])
                    temp[3] = int(temp[3])
                    dist.append(temp)
                else:
                    dist.append(line.split())
            info = dist[:int(dist[0][0])*2+1]
            flights = dist[int(dist[0][0])*2+1:]
        return info, flights
    
    def flights_by_day(self,flight_list):
        # Create an empty dictionary to hold flights organized by day
        flights_by_day = {}

        # Iterate over each flight in the input list
        for flight in flight_list:
            # Extract the day from the flight entry
            day = flight[2]

            # Create a flight entry without the day
            flight_without_day = flight[:2] + flight[3:]

            # Add the flight to the corresponding day in the dictionary
            if day not in flights_by_day:
                flights_by_day[day] = []
            flights_by_day[day].append(flight_without_day)
            
        return flights_by_day
    
    def flights_from_airport(self,flights_by_day, from_airport, considered_day):
        flights_from_airport = []
        for day, flights in flights_by_day.items():
            if day==considered_day:
                for flight in flights:
                    if flight[0] == from_airport:
                        flights_from_airport.append(flight)
                return flights_from_airport
            else:
                return None

    def get_cost(self, day, from_airport, to_airport):
        flights = self.flights_by_day_dict.get(day, [])
        return next(
            (
                flight[2]
                for flight in flights
                if flight[0] == from_airport and flight[1] == to_airport
            ),
            float('inf'),
        )
    
    def get_airports_by_areas(self):
        area_num = int(self.info[0][0])
        return {f"{i}": self.info[2+i * 2] for i in range(0, area_num)}
    
    def associated_area_to_airport(self,airport):
        return next(
            (
                area
                for area, airports in self.airports_by_area.items()
                if airport in airports
            ),
            "Airport not found",
        ) 
    
    def possible_flights_from_an_airport_at_a_specific_day(self,day,from_airport):
        daily_flights = self.flights_by_day_dict.get(day, [])
        
        flights_from_airport = []
        for flight in daily_flights:
            if flight[0] == from_airport:
                
                destination_area = self.associated_area_to_airport(airport=flight[1])
                if destination_area not in self.visited_areas:
                    flights_from_airport.append(flight)

        return flights_from_airport

In [246]:
#print(Data_Preprocessing.starting_airport)
#print(Data_Preprocessing.associated_area_to_airport(airport=Data_Preprocessing.starting_airport))
#print(Data_Preprocessing.visited_areas)
#print(Data_Preprocessing.airports_by_area)
#print(Data_Preprocessing.flights_by_day_dict)

##### First greedy algorithm approach

In [341]:
class heuristics:
    def __init__(self, data_preprocessing_class):
        self.data = data_preprocessing_class
        self.current_airport = self.data.starting_airport
        self.visited_areas = set(self.data.visited_areas)
        self.route = [self.current_airport]
        self.total_cost = 0
    
    
    def cost(self, solution):
        # Calculate the total cost of the provided solution
        total_cost = 0
        for day, (from_airport, to_airport) in enumerate(zip(solution, solution[1:]), start=1):
            total_cost += self.data.get_cost(day, from_airport, to_airport)
        return total_cost
    
    def feasibility(self, solution):
        # Check if the provided solution is feasible
        visited_areas = set()
        for day, (from_airport, to_airport) in enumerate(zip(solution, solution[1:]), start=1):
            area = self.data.associated_area_to_airport(to_airport)
            if area in visited_areas:
                return f"Area in visited areas from {from_airport} to {to_airport}"
            visited_areas.add(area)
            if self.data.get_cost(day, from_airport, to_airport) == float('inf'):
                return f"No existing flight from {from_airport} to {to_airport}"

        # Check if trip starts from the given city and ends in any city of the starting area
        if solution[0] != self.data.starting_airport or self.data.associated_area_to_airport(solution[-1]) != self.data.starting_area:
            return f"Starting city or last city not in good area"

        return len(visited_areas) == self.data.number_of_areas
    
    def create_initial_solution(self):
        # Generate an initial feasible solution using a greedy approach
        for day in range(min(self.data.list_days),max(self.data.list_days)):
            
            next_flight = self.find_next_area(day)
            if not next_flight:
                print("No valid flight found for day", day)
                return
            
            self.route.append(next_flight[1])
            self.total_cost += self.data.get_cost(day, next_flight[0], next_flight[1])
            self.visited_areas.add(self.data.associated_area_to_airport(next_flight[1]))
            self.current_airport = next_flight[1]
            
        # Return to starting area on the last day
        last_day = self.data.number_of_areas
        next_flight = self.find_next_area(last_day)
        if next_flight:
            self.route.append(next_flight[1])
            self.total_cost += self.data.get_cost(last_day, next_flight[0], next_flight[1])
        
        return self.route, self.total_cost
    

    def find_next_area(self, day):
        min_cost = float('inf')
        next_flight = None
        
        flights_from_current = self.data.possible_flights_from_an_airport_at_a_specific_day(day, self.current_airport)
        if not flights_from_current:
            return "No flights at this day"
        
        for flight in flights_from_current:
            cost = self.data.get_cost(day, flight[0], flight[1])
            if cost <= min_cost and self.data.associated_area_to_airport(flight[1]) not in self.visited_areas:
                min_cost = cost
                next_flight = flight
        
        return next_flight
    
    def find_solution(self):
        # Implement a simple local search to improve the initial solution
        best_solution, best_cost = self.create_initial_solution()
        
        # Example of local search: swap two cities in the route and check if the new solution is better
        improved = True
        while improved:
            improved = False
            for i in range(1, len(best_solution) - 1):
                for j in range(i + 1, len(best_solution) - 1):
                    new_solution = best_solution[:]
                    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
                    if self.feasibility(new_solution):
                        new_cost = self.cost(new_solution)
                        if new_cost < best_cost:
                            best_solution, best_cost = new_solution, new_cost
                            improved = True
        return best_solution, best_cost

In [332]:
Data_Preprocessing=data_preprocessing(instance_path="Flight connections dataset/1.in")

In [333]:
Heuristics=heuristics(data_preprocessing_class=Data_Preprocessing)
best_route, best_cost = Heuristics.find_solution()

print("Best Route:", best_route)
print("Best Cost:", best_cost)

Best Route: ['AB0', 'AB1', 'AB8', 'AB4', 'AB5', 'AB3', 'AB7', 'AB2', 'AB9', 'AB6']
Best Cost: 2718


In [335]:
Heuristics.feasibility(solution=['AB0', 'AB1', 'AB8', 'AB4', 'AB9', 'AB2', 'AB7', 'AB3', 'AB5', 'AB6'])

'Starting city or last city not in good area'

In [336]:
Heuristics.cost(['AB0', 'AB1', 'AB8', 'AB4', 'AB5', 'AB3', 'AB7', 'AB2', 'AB9', 'AB6'])

2718

In [338]:
Heuristics.visited_areas

{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [331]:
Data_Preprocessing.associated_area_to_airport('AB8')

'8'

In [327]:
Data_Preprocessing.flights_by_day_dict.get(min(Data_Preprocessing.list_days))

[['AB0', 'AB1', 1],
 ['AB0', 'AB2', 3619],
 ['AB0', 'AB3', 4618],
 ['AB0', 'AB4', 2127],
 ['AB1', 'AB0', 5639],
 ['AB1', 'AB2', 4217],
 ['AB1', 'AB3', 1305],
 ['AB1', 'AB4', 2484],
 ['AB2', 'AB0', 4079],
 ['AB2', 'AB1', 3171],
 ['AB2', 'AB3', 4461],
 ['AB2', 'AB4', 2820],
 ['AB3', 'AB0', 2689],
 ['AB3', 'AB1', 4667],
 ['AB3', 'AB2', 5890],
 ['AB3', 'AB4', 4244],
 ['AB4', 'AB0', 5744],
 ['AB4', 'AB1', 3820],
 ['AB4', 'AB2', 1462],
 ['AB4', 'AB3', 1621],
 ['AB5', 'AB6', 5596],
 ['AB5', 'AB7', 2531],
 ['AB5', 'AB8', 1208],
 ['AB5', 'AB9', 1244],
 ['AB6', 'AB5', 4655],
 ['AB6', 'AB7', 2347],
 ['AB6', 'AB8', 3343],
 ['AB6', 'AB9', 4201],
 ['AB7', 'AB5', 5519],
 ['AB7', 'AB6', 4331],
 ['AB7', 'AB8', 3709],
 ['AB7', 'AB9', 5391],
 ['AB8', 'AB5', 3274],
 ['AB8', 'AB6', 1564],
 ['AB8', 'AB7', 5273],
 ['AB8', 'AB9', 5535],
 ['AB9', 'AB5', 3873],
 ['AB9', 'AB6', 3329],
 ['AB9', 'AB7', 4257],
 ['AB9', 'AB8', 4722],
 ['AB0', 'AB5', 173],
 ['AB5', 'AB0', 200],
 ['AB0', 'AB6', 179],
 ['AB6', 'AB0', 2

##### Backpropagation

In [293]:
class heuristics:
    def __init__(self, data_preprocessing_class):
        self.data = data_preprocessing_class
        self.current_airport = self.data.starting_airport
        self.visited_areas = set(self.data.visited_areas)
        self.route = [self.current_airport]
        self.total_cost = 0
    
    def create_initial_solution(self):
        def backtrack(day):
            if day > self.data.number_of_areas:
                return True
            
            flights_from_current = self.data.possible_flights_from_an_airport_at_a_specific_day(day, self.current_airport)
            if not flights_from_current:
                return False
            
            for flight in flights_from_current:
                next_airport = flight[1]
                next_area = self.data.associated_area_to_airport(next_airport)
                if next_area not in self.visited_areas:
                    self.route.append(next_airport)
                    self.total_cost += self.data.get_cost(day, flight[0], flight[1])
                    self.visited_areas.add(next_area)
                    self.current_airport = next_airport

                    if backtrack(day + 1):
                        return True

                    # Backtrack
                    self.route.pop()
                    self.total_cost -= self.data.get_cost(day, flight[0], flight[1])
                    self.visited_areas.remove(next_area)
                    self.current_airport = flight[0]
            
            return False
        
        # Start backtracking from the first day
        if backtrack(1):
            return self.route, self.total_cost
        else:
            print("No valid initial solution found")
            return None, float('inf')
    
    def cost(self, solution):
        # Calculate the total cost of the provided solution
        total_cost = 0
        for day, (from_airport, to_airport) in enumerate(zip(solution, solution[1:]), start=1):
            total_cost += self.data.get_cost(day, from_airport, to_airport)
        return total_cost
    
    def feasibility(self, solution):
        # Check if the provided solution is feasible
        visited_areas = set()
        for day, (from_airport, to_airport) in enumerate(zip(solution, solution[1:]), start=1):
            area = self.data.associated_area_to_airport(to_airport)
            if area in visited_areas:
                return False
            visited_areas.add(area)
            if self.data.get_cost(day, from_airport, to_airport) == float('inf'):
                return False

        # Check if trip starts from the given city and ends in any city of the starting area
        if solution[0] != self.data.starting_airport or self.data.associated_area_to_airport(solution[-1]) != self.data.starting_area:
            return False

        return len(visited_areas) == self.data.number_of_areas
    
    def find_solution(self):
        # Implement a simple local search to improve the initial solution
        best_solution, best_cost = self.create_initial_solution()
        
        if best_solution is None:
            return None, float('inf')
        
        # Example of local search: swap two cities in the route and check if the new solution is better
        improved = True
        while improved:
            improved = False
            for i in range(1, len(best_solution) - 1):
                for j in range(i + 1, len(best_solution) - 1):
                    new_solution = best_solution[:]
                    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
                    if self.feasibility(new_solution):
                        new_cost = self.cost(new_solution)
                        if new_cost < best_cost:
                            best_solution, best_cost = new_solution, new_cost
                            improved = True
        return best_solution, best_cost
    
    def find_next_area(self, day):
        min_cost = float('inf')
        next_flight = None
        
        flights_from_current = self.data.possible_flights_from_an_airport_at_a_specific_day(day, self.current_airport)
        if not flights_from_current:
            return None
        
        for flight in flights_from_current:
            cost = self.data.get_cost(day, flight[0], flight[1])
            if cost < min_cost and self.data.associated_area_to_airport(flight[1]) not in self.visited_areas:
                min_cost = cost
                next_flight = flight
        
        return next_flight

# Create an instance of the heuristics class
Heuristics = heuristics(data_preprocessing_class=Data_Preprocessing)
best_route, best_cost = Heuristics.find_solution()

print("Best Route:", best_route)
print("Best Cost:", best_cost)

No valid initial solution found
Best Route: None
Best Cost: inf


##### YARO

In [ ]:
def ReadFile(f_name):
    dist = []
    line_nu = -1
    with open(f_name) as infile:
        for line in infile:
            line_nu += 1
            if line_nu == 0:
                index = int(line.split()[0]) * 2 + 1
            if line_nu >= index:
                temp = line.split()
                temp[2] = int(temp[2])
                temp[3] = int(temp[3])
                dist.append(temp)
            else:
                dist.append(line.split())
        info = dist[0:int(dist[0][0])*2+1]
        flights = dist[int(dist[0][0])*2+1:]
    return info, flights

def CreateDistances(File, areas_n):
    mydict = {}
    for i in File:
        if ((i[0],i[1],i[2])) in mydict.keys():
            if mydict[(i[0],i[1],i[2])] > i[3]:
                mydict[(i[0],i[1],i[2])] = i[3]
        else:
            mydict[(i[0],i[1],i[2])] = i[3]
    return mydict

In [ ]:
file="Flight connections dataset/1.in"
File=ReadFile(f_name=file)

In [ ]:
areas_and_airports=File[0][1:]

In [ ]:
dictionnary=CreateDistances(File[1],
                areas_n=int(File[0][0][0]))

In [ ]:
def Initial_Sol(start_air, areas_and_airports, mydict, areas_n):
    start_area = 0
    for i in range(areas_n):
        if start_air in areas_and_airports[i*2 + 1]:
            start_area = i
            break
    solArea = list(range(areas_n + 1))
    Operators.Swap(solArea, 0, start_area)
    solArea[areas_n] = start_area
    
    solAirport = [start_air]
    for i in range(1,areas_n + 1):
        solAirport.append(areas_and_airports[solArea[i]*2 + 1][0])
    
    return solArea, solAirport